In [ ]:
import sys
print(sys.executable)

In [ ]:
# dependecies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
import random
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import xgboost as xgb
from xgboost import XGBClassifier

In [ ]:
# for all my plots
palette = ['#C66F80', '#F4C7D0','#FCEBF1', '#4A6644', '#9FAA74', '#D7DAB3', '#ECE3D2']
customCmap = ListedColormap(palette)

In [ ]:
data = pd.read_csv('personalFinanceDataset.csv')
data.head()

In [ ]:
data.info()

In [ ]:
data = data.drop(['Unnamed: 0'], axis=1)

In [ ]:
data.head()

In [ ]:
data.isnull().sum()

In [ ]:
data.duplicated().sum()

In [ ]:
data.shape

In [ ]:
# making all categorical values numerical => correlation table (one hot encoding)
ohe_gender = pd.get_dummies(data.gender)
ohe_raceEthnicity = pd.get_dummies(data.raceEthnicity)
ohe_education = pd.get_dummies(data.education)
ohe_region = pd.get_dummies(data.region)
ohe_householdType = pd.get_dummies(data.householdType)
ohe_career = pd.get_dummies(data.career)
ohe_workArrangement = pd.get_dummies(data.workArrangement)
ohe_housingStatus = pd.get_dummies(data.housingStatus)

In [ ]:
data = pd.concat([data, ohe_gender, ohe_career, ohe_education, ohe_householdType, ohe_housingStatus, ohe_raceEthnicity, ohe_region, ohe_workArrangement], axis='columns')
data.head()

writing down some things i want to explore
 1. want to see correlations in different categories
 - more specific one : debt & discretionary
 2. want to see feature importance (which feature has the biggest impact on financial status)
 3. think wanted to learn xg boost model - def want to build a model anyway to simulate real people
 - wanna experiment on lesser features used to train; bc not everyone would have all these numbers ready to go
 4. are there any instances that break the original proportions

In [ ]:
df = data[['studentLoans', 'entertainment', 'hobbies', 'travel', 'gifts', 'donations', 'ccPayment', 'personalLoans', 'medicalDebt']].copy()
df.head()

In [ ]:
correlationMatrix = df.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlationMatrix, annot=True, cmap=customCmap, 
            fmt='.1f', square=True, linewidths=0.5)
plt.title('Correlation Matrix between Debt & Discretionary Spending',)
plt.tight_layout()
plt.show()

In [ ]:
# creating some cross tables
crosstab01 = pd.crosstab(data['career'], data['financialHealth'])
crosstab02 = pd.crosstab(data['financialHealth'], data['householdType'])
crosstab03 = pd.crosstab(data['ageCategory'], data['career'])
crosstab04 = pd.crosstab(data['financialHealth'], data['career'])
crosstab04

In [ ]:
crosstab02.plot(kind='bar', stacked=True, colormap=customCmap, figsize=(12, 6))

In [ ]:
# about to begin process of using xgboost vastly for feature importance data
# must first label encode y variable
le = LabelEncoder()
y = le.fit_transform(data['financialHealth']) # works for the target variable, not for features

In [ ]:
data.select_dtypes(include='object').info()

In [ ]:
# test train split
x = data.drop(['gender','raceEthnicity', 'education', 'region', 'householdType',
              'career', 'workArrangement', 'housingStatus', 'financialHealth',
              'ageCategory', 'totalExpenses', 'totalSavingsInvestments', 'monthlyCashFlow', 'housingRatio', 'debtToIncome', 'savingsInvestmentsRate', 'carPaymentRatio', 'monthsSaved'], axis=1)
featureNames = x.columns
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
yTrain = yTrain.flatten()
yTrain.shape

In [ ]:
xTrain = xTrain.astype('float32').values
xTest = xTest.astype('float32').values
yTrain = yTrain.astype('int')

In [ ]:
# model
model = XGBClassifier(random_state=42, n_estimators=100, max_depth=4)
model.fit(xTrain, yTrain)

In [ ]:
# evaluate
yPred = model.predict(xTest)
print(classification_report(yTest, yPred))

In [ ]:
print(le.classes_)

In [ ]:
# confusion matrix
cm = confusion_matrix(yTest, yPred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap=customCmap, xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Confusion Matrix', fontsize = 16, fontweight = 'bold')
plt.xlabel('Predicted Label', fontsize = 12)
plt.ylabel('True Label', fontsize = 12)
plt.tight_layout()

In [ ]:
# feature importance
xgb.plot_importance(model, max_num_features=10)
plt.show()

In [ ]:
# Get feature importances as a dictionary
importances = model.get_booster().get_score(importance_type='weight')

# Match them to the real column names
feature_map = {f"f{i}": col for i, col in enumerate(featureNames)}

# Translate importances
named_importances = {feature_map.get(k, k): v for k, v in importances.items()}

# Sort and view
import pandas as pd
pd.Series(named_importances).sort_values(ascending=False)

i want to do a KNN model & test out the differences in my synthetic data made using a python script vs using a GAN and seeing the similarities & differences in the XG Boost & KNN models

In [ ]:
# test train split
x = data[['housingCost', 'monthlyIncome', 'emergencyFundContributions', 'carPayment', 
           'ccPayment', 'clothing', 'streaming', 'generalSavings', 'gifts', 
           'personalCare', 'entertainment', 'alcohol', 'age', 'phone',
            'healthInsurance', 'travel', 'oopMedical', 'householdSupplies', 
            'coffee', 'homeInsurance', 'internet', 'groceries']].copy()
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
xTrain = scaler.fit_transform(xTrain)
xTest = scaler.transform(xTest)

In [ ]:
kRange = range(5, 25)
cvScores = []

for k in kRange:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, xTrain, yTrain, cv=5, scoring='f1_macro')
    cvScores.append(scores.mean())

In [ ]:
plt.plot(kRange, cvScores, marker='o')
plt.xlabel('k')
plt.ylabel('Cross-Validated Accuracy')
plt.title('Optimal k for KNN')
plt.grid(True)
plt.show()

best_k = kRange[cvScores.index(max(cvScores))]
print(f"Best k: {best_k}")

In [ ]:
knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(xTrain, yTrain)

In [ ]:
yPred = knn.predict(xTest)

In [ ]:
# evaluate
print(classification_report(yTest, yPred))

In [ ]:
# confusion matrix
cm = confusion_matrix(yTest, yPred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap=customCmap, xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Confusion Matrix', fontsize = 16, fontweight = 'bold')
plt.xlabel('Predicted Label', fontsize = 12)
plt.ylabel('True Label', fontsize = 12)
plt.tight_layout()